In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import xgboost as xgb
import sklearn.metrics as metrics
import datetime

In [2]:
inputfolder = 'data'
datadf = pd.read_csv(os.path.join(inputfolder,"mens_match_data.csv"))
datadf['datetime'] = pd.to_datetime(datadf['datetime'])
with open(os.path.join(inputfolder,'datacolumns.txt'),'r') as f:
    content = f.readlines()
datacols = [x.strip() for x in content]
print(datadf.dtypes)
display(datadf.head())

atpdata = pd.read_csv(os.path.join(inputfolder,"ATP_matches.csv"))
cols = atpdata.drop(['Winner','Loser','Tournament','Tournament_Date','Court_Surface','Round_Description'],axis=1).columns
for col in cols:
    atpdata.loc[:,col]=pd.to_numeric(atpdata[col],errors='coerce')
print(atpdata.dtypes)
atpdata['datetime'] = pd.to_datetime(atpdata['Tournament_Date'])
atpdata['year'] = pd.DatetimeIndex(atpdata['datetime']).year
def fillrank(row,col,ATPframe):
    if np.isnan(row[col]):
            year = row['year']
            tournament = row['Tournament']
            surface = row['Court_Surface']
            yeardf = ATPframe[ATPframe['year']==year] #i can do this slicing in one line, but im doing in two to allow for easier debugging
            tourndf = ATPframe[(ATPframe['Tournament']==tournament)&(ATPframe['Court_Surface']==surface)]
            maxrank = int(max([tourndf['Winner_Rank'].max(),tourndf['Loser_Rank'].max()]))
            return maxrank+1
    else:
        return row[col]
atpdata.loc[:,'Winner_Rank'] = atpdata.apply(fillrank,axis=1,args=('Winner_Rank',atpdata))
atpdata.loc[:,'Loser_Rank'] = atpdata.apply(fillrank,axis=1,args=('Loser_Rank',atpdata))

display(atpdata.head())

#only going to use matches from 2014 onwards, not ussing matches from 2012-13 so we have enough historic data for matches in 2014
atpHardOnly = atpdata.loc[(atpdata['Court_Surface']=='Hard')&(atpdata['year']>2013),['Winner','Loser','datetime']]
frame1,frame2 = train_test_split(atpHardOnly,test_size=.5)
frame1.columns = ['Player1','Player2','datetime']
frame1['Player1Win'] = 1

frame2.columns = ['Player2','Player1','datetime']
frame2['Player1Win'] = 0

traininput = pd.concat([frame1,frame2])
traininput.sort_index(inplace=True)
print(traininput['Player1Win'].mean())
display(traininput.head())

# splitting into X and Y
X = traininput.drop('Player1Win',axis=1)
Y = traininput['Player1Win']

Player                        object
Rank                         float64
Sets_Won                     float64
Games_Won                      int64
Aces                         float64
DoubleFaults                 float64
FirstServes_Won              float64
FirstServes_In               float64
SecondServes_Won             float64
SecondServes_In              float64
BreakPoints_Won              float64
BreakPoints                  float64
ReturnPoints_Won             float64
ReturnPoints_Faced           float64
TotalPoints_Won                int64
won_game?                      int64
FirstServes_ratio            float64
SecondServes_ratio           float64
BreakPoints_ratio            float64
ReturnPoints_ratio           float64
datetime              datetime64[ns]
year                           int64
Tournament                    object
Round_Description             object
Court_Surface                 object
Total_Serves                 float64
Aces%                        float64
S

,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface,Total_Serves,Aces%,ServesWon%
0,Edouard Roger-Vasselin,106.0,2.0,12,5.0,2.0,22.0,30.0,12.0,19.0,...,0.571429,0.423729,2012-01-02,2012,Chennai,First Round,Hard,49.0,0.102041,0.693878
1,Dudi Sela,83.0,2.0,12,2.0,0.0,14.0,17.0,11.0,16.0,...,0.428571,0.620690,2012-01-02,2012,Chennai,First Round,Hard,33.0,0.060606,0.757576
2,Go Soeda,120.0,2.0,19,6.0,1.0,48.0,64.0,19.0,39.0,...,0.454545,0.400000,2012-01-02,2012,Chennai,First Round,Hard,103.0,0.058252,0.650485
3,Yuki Bhambri,345.0,2.0,12,1.0,2.0,22.0,29.0,9.0,17.0,...,0.384615,0.548387,2012-01-02,2012,Chennai,First Round,Hard,46.0,0.021739,0.673913
4,Yuichi Sugita,235.0,2.0,12,3.0,1.0,37.0,51.0,11.0,27.0,...,0.428571,0.407407,2012-01-02,2012,Chennai,First Round,Hard,78.0,0.038462,0.615385


C:\Users\Hugh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,12,13,14,17,18,24,25,26,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Winner                        object
Loser                         object
Tournament                    object
Tournament_Date               object
Court_Surface                 object
Round_Description             object
Winner_Rank                  float64
Loser_Rank                   float64
Retirement_Ind                 int64
Winner_Sets_Won              float64
Winner_Games_Won               int64
Winner_Aces                  float64
Winner_DoubleFaults          float64
Winner_FirstServes_Won       float64
Winner_FirstServes_In        float64
Winner_SecondServes_Won      float64
Winner_SecondServes_In       float64
Winner_BreakPoints_Won       float64
Winner_BreakPoints           float64
Winner_ReturnPoints_Won      float64
Winner_ReturnPoints_Faced    float64
Winner_TotalPoints_Won         int64
Loser_Sets_Won               float64
Loser_Games_Won                int64
Loser_Aces                   float64
Loser_DoubleFaults           float64
Loser_FirstServes_Won        float64
L

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,...,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won,datetime,year
0,Edouard Roger-Vasselin,Eric Prodon,Chennai,02-Jan-12,Hard,First Round,106.0,97.0,0,2.0,...,33.0,13.0,26.0,1.0,3.0,15.0,49.0,49,2012-01-02,2012
1,Dudi Sela,Fabio Fognini,Chennai,02-Jan-12,Hard,First Round,83.0,48.0,0,2.0,...,32.0,5.0,26.0,0.0,1.0,8.0,33.0,30,2012-01-02,2012
2,Go Soeda,Frederico Gil,Chennai,02-Jan-12,Hard,First Round,120.0,102.0,0,2.0,...,70.0,18.0,35.0,2.0,4.0,36.0,103.0,99,2012-01-02,2012
3,Yuki Bhambri,Karol Beck,Chennai,02-Jan-12,Hard,First Round,345.0,101.0,0,2.0,...,33.0,13.0,29.0,2.0,3.0,15.0,46.0,43,2012-01-02,2012
4,Yuichi Sugita,Olivier Rochus,Chennai,02-Jan-12,Hard,First Round,235.0,67.0,0,2.0,...,32.0,13.0,22.0,1.0,7.0,30.0,78.0,62,2012-01-02,2012


0.4999257609502598


C:\Users\Hugh\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hugh\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hugh\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=Fals

,Player1,Player1Win,Player2,datetime
5893,Edouard Roger-Vasselin,1,Albert Ramos-Vinolas,2014-01-01
5894,Aleksandr Nedovyesov,0,Guillermo Garcia-Lopez,2014-01-01
5895,Aljaz Bedene,1,Henri Laaksonen,2014-01-01
5896,Jeevan Nedunchezhiyan,0,Jiri Vesely,2014-01-01
5897,Benjamin Becker,1,Julian Reister,2014-01-01


In [3]:
#defining preprocessing functions (from wrangleDataAttempt2)
def days_difference(date1,date2):
            diff = date2-date1
            return diff.days

def get_player_stats(inputframe,harddf,datacols,playercol):
    #idea is to pass a panda dataframe with columns [playername, tournamentdate] and be able to return a dataframe with the stats
    length = inputframe.shape[0]
    historicnames = ['careeravg_'+x for x in datacols]
    ytdnames = ['ytd_'+x for x in datacols]
    colnames = [playercol,'datetime','Rank','ytd_Aces%_Stddev','ytd_ServesWon%_Stddev'] + historicnames+ytdnames
    outputframe = pd.DataFrame(index=range(0,length),columns=colnames)
    outputframe.loc[:,playercol] = inputframe[playercol]
    outputframe.loc[:,'datetime'] = inputframe['datetime']
    for index in range(0,length):
        #extract player name and date of game
        row = inputframe.iloc[index,:]
        playername = row[playercol]
        date = row['datetime']
        
        #grab only data for that player before that date
        tempdf = harddf[harddf['Player']==playername]
        tempdf.loc[:,'timedelta'] = tempdf['datetime'].apply(days_difference,args=(date,))
        tempdf = tempdf[tempdf['timedelta']>0]
        if tempdf.empty:
            continue
        ytddf = tempdf.loc[tempdf['timedelta']<=365]
#         display(ytddf.head())
        currank = tempdf.loc[tempdf['timedelta'].idxmin(),'Rank']
        historicframe = tempdf.loc[:,datacols].mean()
        historicframe.index = historicnames
        YTDframe = ytddf.loc[:,datacols].mean()
        YTDframe.index = ytdnames
        
        #code for debugging NaNs
#         if tempdf.isna().sum().sum()>0:
#             print(playername)
#             print("ytdframe shape: {}, missing values: {}".format(ytddf.shape,ytddf.isna().sum().sum()))
#             print(ytddf.isna().sum())
#             print("careerdf shape: {}, missing values: {}".format(tempdf.shape,tempdf.isna().sum().sum()))
#             print(tempdf.isna().sum())

        outputframe.loc[index,['Rank','ytd_Aces%_Stddev','ytd_ServesWon%_Stddev']] = [currank,ytddf['Aces%'].std(),ytddf['ServesWon%'].std()]
        outputframe.loc[index,historicnames]=historicframe
        outputframe.loc[index,ytdnames]=YTDframe
    return outputframe

def get_difference(frame1,frame2,colname=None, prefix1='',prefix2='',index1=None,index2=None):
    #check if index between frames are identical, if not, reset both
    idx_check = frame1.index.equals(frame2.index)
    if not(idx_check):
        frame1.reset_index(drop=True, inplace=True)
        frame2.reset_index(drop=True, inplace=True)
    if colname is None:
        colname = frame1.columns
    if frame1.shape[0] != frame2.shape[0]:
        raise ValueError('both frames must contain the same number of rows/n Frame1: %{}/mn Frame2: %{}'.format(frame1.shape[0],frame2.shape[0]))
    tempdiffcolnames = [x+"_diff" for x in colname]
    diffcolnames = list()
    if index1 is not None:
        diffcolnames.append(index1)
    if index2 is not None:
        diffcolnames.append(index2)
    diffcolnames = diffcolnames + tempdiffcolnames
    if idx_check:
        difframe = pd.DataFrame(index=frame1.index,columns=diffcolnames,data=0)
    else:
        difframe = pd.DataFrame(index=range(0,frame1.shape[0]),columns=diffcolnames,data=0)
    if index1 is not None:
        difframe[index1] = frame1[index1]
    if index2 is not None:
        difframe[index2] = frame2[index2]
    for col in colname:
        frame1name = prefix1+col
        frame2name = prefix2+col
        difframename = col+"_diff"
        difframe[difframename]=frame1[frame1name] - frame2[frame2name]
    return difframe

def convert_to_data(inputframe,matchdata,datacols,player1colname="Player1",player2colname="Player2"):
    inputframe.reset_index(drop=True,inplace=True)
    
    player1frame = inputframe.loc[:,[player1colname,'datetime']]
    player2frame = inputframe.loc[:,[player2colname,'datetime']]
    
    player1df = get_player_stats(player1frame,matchdata,datacols,player1colname)
    player2df = get_player_stats(player2frame,matchdata,datacols,player2colname)
    
    #append columns indicating whether this is a players first recorded Hard surface ATPgame (if it's their first game, Rank will return NaNs)
    player1df['Player1fg'] = player1df['Rank'].isna()
    player2df['Player2fg'] = player2df['Rank'].isna()
    
#     player1df = player1df[~player1df['Player1fg']].fillna(0)
#     player2df = player2df[~player2df['Player2fg']].fillna(0)
#     if player1df.isna().sum().sum() >0:
#         display(player1df)
    
    
    #for debugging purposes
    if player1df.shape[0] != player2df.shape[0]:
        return (player1df,player2df)
    
    newdatacols = player1df.drop([player1colname,'datetime','Player1fg'],axis=1).columns
    outputdf = get_difference(player1df,player2df,newdatacols,index1=player1colname,index2=player2colname)
    outputdf.loc[:,'Player1fg'] = player1df['Player1fg']
    outputdf.loc[:,'Player2fg'] = player2df['Player2fg']
    outputdf.loc[:,'datetime'] = player1df['datetime']
    
    return outputdf.infer_objects() #infer_objects soft converts object columns to their correct types

# tempX = X.iloc[:15,:]
# tempX = convert_to_data(tempX,datadf,datacols)
# print(tempX.dtypes)

X = convert_to_data(X,datadf,datacols)
print(X.dtypes)

C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Player1                                      object
Player2                                      object
Rank_diff                                   float64
ytd_Aces%_Stddev_diff                       float64
ytd_ServesWon%_Stddev_diff                  float64
careeravg_Sets_Won_diff                     float64
careeravg_Games_Won_diff                    float64
careeravg_Aces_diff                         float64
careeravg_DoubleFaults_diff                 float64
careeravg_FirstServes_Won_diff              float64
careeravg_FirstServes_In_diff               float64
careeravg_SecondServes_Won_diff             float64
careeravg_SecondServes_In_diff              float64
careeravg_BreakPoints_Won_diff              float64
careeravg_BreakPoints_diff                  float64
careeravg_ReturnPoints_Won_diff             float64
careeravg_ReturnPoints_Faced_diff           float64
careeravg_TotalPoints_Won_diff              float64
careeravg_won_game?_diff                    float64
careeravg_Fi

In [4]:
#training and then testing a basic model
from sklearn.model_selection import GridSearchCV
fullX = X.copy()
if 'Player1' in X.columns:
    X.drop(['Player1','Player2','datetime'],axis=1,inplace=True)
Xtrain,Xtest,ytrain,ytest = train_test_split(X,Y,test_size=.3)
params ={'max_depth':[3,4,5],
         'n_estimators':[100,200,300],
         'learning_rate':[.01,.03,.1],
         'reg_alpha':[0,.01,.03],
         'reg_lambda':[0,.01,.03]
        }
estimator = xgb.XGBClassifier(random_state=123)
tuneparam = GridSearchCV(estimator,params,scoring='log_loss',n_jobs=-1,cv=5)
print("tuning model")
tuneparam.fit(Xtrain,ytrain)

model = tuneparam.best_estimator_
best_params = tuneparam.best_params_
cvtable = tuneparam.cv_results_
cvdf = pd.DataFrame(cvtable)
display(cvdf[cvdf['rank_test_score']==1])

def get_scores(predy,ytest):
    import sklearn.metrics as metrics
    acc = metrics.accuracy_score(ytest,predy)
    prec = metrics.precision_score(ytest,predy)
    recall = metrics.recall_score(ytest,predy)
    auc = metrics.roc_auc_score(ytest,predy)
    return {'accuracy':acc,'precision':prec,'recall':recall,'auc':auc}
score = model.score(Xtest,ytest)
predy = model.predict(Xtest)
proby = model.predict_proba(Xtest)
print(proby.shape)

scoresdict = get_scores(predy,ytest)
logloss = metrics.log_loss(ytest,proby)
print("="*200)
print("Score: {}".format(score))
print("logloss: {}".format(logloss))
for key,value in scoresdict.items():
    print("{}: {}".format(key,value))

print("="*200)
for idx, col in enumerate(Xtrain.columns.tolist()):
    print("{} importance: {}".format(col,model.feature_importances_[idx]))


tuning model


C:\Users\Hugh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Hugh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Hugh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Hugh\Anaconda3\lib\site-packages\sklearn\utils\de

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_reg_alpha,param_reg_lambda,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
91,7.762243,0.03074,0.012167,0.000977,0.03,3,200,0,0.01,"{'learning_rate': 0.03, 'max_depth': 3, 'n_est...",...,-0.615267,0.004893,1,-0.557342,-0.550365,-0.54699,-0.555851,-0.553275,-0.552764,0.003737


C:\Users\Hugh\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Hugh\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Score: 0.6605640771895102
logloss: 0.6253904204157571
accuracy: 0.6605640771895102
precision: 0.6829015544041451
recall: 0.6342637151106834
auc: 0.6613273769036105
Rank_diff importance: 0.08487654477357864
ytd_Aces%_Stddev_diff importance: 0.03163580223917961
ytd_ServesWon%_Stddev_diff importance: 0.015432098880410194
careeravg_Sets_Won_diff importance: 0.06635802239179611
careeravg_Games_Won_diff importance: 0.03780864179134369
careeravg_Aces_diff importance: 0.019290123134851456
careeravg_DoubleFaults_diff importance: 0.014660493470728397
careeravg_FirstServes_Won_diff importance: 0.004629629664123058
careeravg_FirstServes_In_diff importance: 0.021604938432574272
careeravg_SecondServes_Won_diff importance: 0.0038580247201025486
careeravg_SecondServes_In_diff importance: 0.004629629664123058
careeravg_BreakPoints_Won_diff importance: 0.01774691417813301
careeravg_BreakPoints_diff importance: 0.018518518656492233
careeravg_ReturnPoints_Won_diff importance: 0.03703703731298447
careeravg

What about a model just trained using Ranked Diff? (curious to see how much predictive power rank has)

In [5]:
rankXtrain = Xtrain.loc[:,'Rank_diff'].values.reshape(Xtrain.shape[0],1)
rankXtest = Xtest.loc[:,'Rank_diff'].values.reshape(Xtest.shape[0],1)

# print(rankXtrain.shape)
params ={'max_depth':[4,5,6],
         'n_estimators':[100,200,300],
         'learning_rate':[.01,.03,.1],
        }
estimators = xgb.XGBClassifier(random_state=123)
tuneparams = GridSearchCV(estimators,params,scoring='neg_log_loss',n_jobs=-1)
print("tuning model")
tuneparams.fit(rankXtrain,ytrain)

model2 = tuneparams.best_estimator_
best_params2 = tuneparams.best_params_

score2 = model2.score(rankXtest,ytest)
predy2 = model2.predict(rankXtest)
scoresdict2 = get_scores(predy2,ytest)

print("="*200)
print("Score: {}".format(score2))
for key,value in scoresdict2.items():
    print("{}: {}".format(key,value))

print("="*200)
print("{} importance: {}".format('Rank_diff',model2.feature_importances_))


tuning model
Score: 0.6338446313706086
accuracy: 0.6338446313706086
precision: 0.6598930481283423
recall: 0.5938402309913379
auc: 0.6350056552105366
Rank_diff importance: [1.]


C:\Users\Hugh\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Hugh\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [6]:
print(best_params)
print(best_params2)

{'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 200, 'reg_alpha': 0, 'reg_lambda': 0.01}
{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 200}


In [16]:
# final_submission = pd.read_csv('men_final_submission_file.csv')
# submissiondata = final_submission.loc[:,['player_1','player_2']]
# submissiondata['datetime'] = datetime.datetime.now()
# submissionX = convert_to_data(submissiondata,datadf,datacols,'player_1','player_2')
display(submissionX.head())

predprob = model.predict_proba(submissionX.drop(['player_1','player_2','datetime'],axis=1))
display(predprob)
print(final_submission.shape)
final_submission.loc[:,'player_1_win_probability'] = predprob[:,1]
final_submission.to_csv('mens_submission.csv',index=False)

,player_1,player_2,Rank_diff,ytd_Aces%_Stddev_diff,ytd_ServesWon%_Stddev_diff,careeravg_Sets_Won_diff,careeravg_Games_Won_diff,careeravg_Aces_diff,careeravg_DoubleFaults_diff,careeravg_FirstServes_Won_diff,...,ytd_FirstServes_ratio_diff,ytd_SecondServes_ratio_diff,ytd_BreakPoints_ratio_diff,ytd_ReturnPoints_ratio_diff,ytd_Total_Serves_diff,ytd_Aces%_diff,ytd_ServesWon%_diff,Player1fg,Player2fg,datetime
0,Novak Djokovic,Mitchell Krueger,-230,0.015845,0.011234,0.892937,2.499628,0.403656,-0.722825,2.435836,...,0.054056,0.105534,0.192201,0.032683,-11.819444,0.017964,0.084637,False,False,2019-01-18 16:44:22.363044
1,Novak Djokovic,Jo-Wilfried Tsonga,-238,0.011612,-0.006988,0.373512,0.432002,-4.714450,-0.703832,-2.070926,...,-0.027720,0.123789,-0.142298,0.072733,6.305556,-0.105459,0.039589,False,False,2019-01-18 16:44:22.363044
2,Novak Djokovic,Martin Klizan,-39,0.032916,-0.044253,1.009603,3.024628,0.084211,-2.282270,2.545281,...,0.074822,0.128695,0.204672,0.067051,-21.444444,0.053896,0.104559,False,False,2019-01-18 16:44:22.363044
3,Novak Djokovic,Denis Shapovalov,-26,-0.001082,0.000547,0.572103,0.566295,-2.304678,-4.046159,-1.753330,...,0.003768,0.089087,0.025630,0.049659,-10.259259,-0.020979,0.057036,False,False,2019-01-18 16:44:22.363044
4,Novak Djokovic,David Goffin,-21,0.013201,-0.030864,0.623138,1.825803,0.097727,-2.070208,2.381877,...,0.045634,0.085465,0.086922,0.027273,1.993056,0.019716,0.065418,False,False,2019-01-18 16:44:22.363044


array([[0.12310123, 0.87689877],
       [0.19995719, 0.8000428 ],
       [0.09414136, 0.90585864],
       ...,
       [0.12703782, 0.8729622 ],
       [0.15296346, 0.84703654],
       [0.1487261 , 0.8512739 ]], dtype=float32)

(8128, 3)
